In [1]:
import json
import matplotlib.pyplot as plt
from keras.utils import to_categorical
from tqdm import tqdm
import random
import numpy as np

Using TensorFlow backend.


In [2]:
%matplotlib inline

In [3]:
merchant_id = '196E2517293A9285'

In [4]:
file = '/data/predict/'+merchant_id+'/'+merchant_id+'_logs.json'

In [5]:
users = {}
with open(file, encoding='utf8') as f:
    for line in tqdm(f):
        data = json.loads(line)
        if data['visitor_id'] not in users:
            users[data['visitor_id']] = {'visits':[], 'cart':[]}
            
        for prod_idx in data['viewed_products']:
            try:
                users[data['visitor_id']]['visits'].append(prod_idx)
            except ValueError: 
                print('Skiped ' + prod_idx)
                
        for prod_idx in data['cart']:
            try:
                users[data['visitor_id']]['cart'].append(prod_idx)
            except ValueError: 
                print('Skiped ' + prod_idx)
        

759250it [00:06, 115675.38it/s]


In [6]:
count = 0
for user_id,user in users.items():
    
    if len(user['cart']) > 0:
        count += 1 
count

9969

In [7]:
products = {}
for u in users.values():
    for pid in u['visits']:
        products[pid] = 1
        
    for pid in u['cart']:
        products[pid] = 1
        
product_idxs = products.keys()
product_ids = {v: k for k, v in enumerate(product_idxs)}
number_of_classes = len(product_idxs)

In [8]:
print(number_of_classes)

2425


In [9]:
def chunks(l, n):
    for i in range(0, len(l), n):
        yield l[i:i + n]

In [10]:
def dedup(seq):
    seen = set()
    seen_add = seen.add

    return [x for x in seq if not (x in seen or seen_add(x))]


users_with_product_idxs = {}
for uid,sessions in users.items():
    visit_ids = dedup(list(map(lambda x: product_ids[x], sessions['visits'])))[-40:]
    cart_ids = dedup(list(map(lambda x: product_ids[x], sessions['cart'])))[-40:]
    if len(cart_ids) > 0 and len(visit_ids):
        if len(visit_ids) not in users_with_product_idxs:
            users_with_product_idxs[len(visit_ids)] = []
        users_with_product_idxs[len(visit_ids)].append({
            'visits': visit_ids,
            'cart': cart_ids
        })
    
#del users

# TODO
Bellow here nothing has been updated yet

In [13]:
max_batch_size = 32
test_size = 0.1
user_batches = []
user_valid_batches = []
for len_group in users_with_product_idxs.values():
    random.shuffle(len_group)
    for batch in chunks(len_group, max_batch_size):
        batch_data = np.array(list(map(lambda x: x[:-1], batch)))
        batch_labels = np.expand_dims(np.array(list(map(lambda x: x[1:], batch))), axis=2)
        if random.random() < test_size:
            user_valid_batches.append(
                (batch_data, batch_labels)
            )
        else:
            user_batches.append(
                (batch_data, batch_labels)
            )

In [14]:

def generate_from_array(arr):
    while True:
        random.shuffle(arr)
        for i in arr: 
            yield i
    

In [24]:
from keras.models import Sequential
from keras.layers import Embedding, CuDNNGRU, Dense, Dropout, TimeDistributed,Bidirectional
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Nadam
from keras.optimizers import Nadam


n_hidden = 200

model=Sequential([
        Embedding(number_of_classes, n_hidden, batch_input_shape=(None,None)),
        BatchNormalization(),
        CuDNNGRU(4*n_hidden, input_shape=(None,n_hidden),return_sequences=True),
        Dropout(0.2),
        CuDNNGRU(4*n_hidden,return_sequences=True),
        Dropout(0.2),
        TimeDistributed(Dense(n_hidden, activation='relu')),
        TimeDistributed(Dense(number_of_classes, activation='softmax'))
    ])

In [25]:
from keras.backend import reshape, prod, shape, stack
from keras.metrics import sparse_top_k_categorical_accuracy

def flatten_first_n_dimension(x, first_n=2):
    new_dims = stack([prod(shape(x)[:first_n]), prod(shape(x)[first_n:])])

    return reshape(x, new_dims)

def sparse_timedistributed_top_k_categorical_accuracy(y_true, y_pred, k=5):
    y_true_reshaped = flatten_first_n_dimension(y_true)
    y_pred_reshaped = flatten_first_n_dimension(y_pred)

    return sparse_top_k_categorical_accuracy(y_true_reshaped, y_pred_reshaped, k)

In [26]:
model.compile(loss='sparse_categorical_crossentropy', metrics=['sparse_categorical_accuracy',sparse_timedistributed_top_k_categorical_accuracy], optimizer=Nadam())

In [27]:
import csv
with open('/data/predict//'+merchant_id+'/'+merchant_id+'_variable_length_model_id_map.csv', 'w') as csvfile:
    spamwriter = csv.writer(csvfile)
    for line in product_idxs:
        spamwriter.writerow([line])

In [28]:
from keras.callbacks import ModelCheckpoint
filepath='/data/predict/'+merchant_id+'/'+merchant_id+'_variable_length_model-{epoch:02d}-{val_loss:.2f}.hdf5'
checkpoint = ModelCheckpoint(filepath, monitor='val_sparse_timedistributed_top_k_categorical_accuracy', verbose=1, save_best_only=False, mode='max')

In [29]:
model.fit_generator(
    generate_from_array(user_batches), 
    validation_data=generate_from_array(user_valid_batches), 
    steps_per_epoch=len(user_batches),
    validation_steps=len(user_valid_batches),
    epochs=6,
    callbacks=[checkpoint]
)

Epoch 1/6
2034/2034 [==============================] - 29s 14ms/step - loss: 5.6848 - sparse_categorical_accuracy: 0.1111 - sparse_timedistributed_top_k_categorical_accuracy: 0.2554 - val_loss: 5.3079 - val_sparse_categorical_accuracy: 0.1460 - val_sparse_timedistributed_top_k_categorical_accuracy: 0.3163
Epoch 2/6
2034/2034 [==============================] - 28s 14ms/step - loss: 5.1494 - sparse_categorical_accuracy: 0.1517 - sparse_timedistributed_top_k_categorical_accuracy: 0.3252 - val_loss: 5.1990 - val_sparse_categorical_accuracy: 0.1623 - val_sparse_timedistributed_top_k_categorical_accuracy: 0.3344
Epoch 3/6
2034/2034 [==============================] - 27s 13ms/step - loss: 4.9926 - sparse_categorical_accuracy: 0.1638 - sparse_timedistributed_top_k_categorical_accuracy: 0.3426 - val_loss: 5.1525 - val_sparse_categorical_accuracy: 0.1721 - val_sparse_timedistributed_top_k_categorical_accuracy: 0.3506
Epoch 4/6
2034/2034 [==============================] - 28s 14ms/step - loss: 4.

## Eye ball validation

In [43]:
import csv

In [44]:
product_map = {}
with open('/data/predict/'+merchant_id+'/'+merchant_id+'_catalog.csv', 'r', encoding='utf8') as csvfile:
    spamreader = csv.reader(csvfile)
    header = next(spamreader)
    item_id = header.index('item')
    image_url = header.index('image')
    title = header.index('title')
    link = header.index('link')
    
    for row in spamreader:
        product_map[row[item_id]] = [row[image_url], row[title], row[link]]

In [46]:
from IPython.core.display import display as core_display
from IPython.core.display import HTML 

def product_to_url(product):
    try:
        
        return [product] + product_map[list(product_idxs)[product]] + [list(product_idxs)[product]]
    except KeyError:
        return [product,'http://www.lackuna.com/wp-content/themes/fearless/images/missing-image-640x360.png','UNKNOWN','', '']

def display_session(session):
    products = list(map(product_to_url, session))
    predictions = model.predict([session])
    html = '<table>'
    seen = set()
    for i,product in enumerate(products):
        html += '<tr>'
        html += '<th style="text-align:center">'
        html += '<img src="' + product[1] + '" width="130"><br/>' 
        html += '<a href="'+str(product[3])+'">' + str(product[4]) + '</a></br>' + str(product[2])
        html += '</th><td style="background-color:black;"></td>'
        top_idx = list(np.argsort(predictions[i][0])[::-1])
        seen.add(product[0])
        
        j = 0
        for pred in top_idx:
            if pred in seen:
                continue
                                
            if i < len(products) - 1 and pred == products[i+1][0]:
                style = 'style="border:3px dashed red;"'
            else:
                style = ''
                
            pred_data = product_to_url(pred)
            html += '<td><img src="' + pred_data[1] + '" '+style+' width="130"><br/>' 
            html += '<a href="'+str(pred_data[3]) + '">' +str(pred_data[4]) + '</a></br>' + str(pred_data[2]) + '</td>'
            j+=1
            if j > 5:
                break
                
        html += '</tr>'
    html += '</table>'

    core_display(HTML(html))

display_session(random.choice(random.choice(user_valid_batches)[0]))

A37096411053Coated Slim Fit Hose Yulius - master wine dark,,A37096410001Coated Slim Fit Hose Yulius - Black,A37096410109Coated Slim Fit Hose Yulius - night blue,A37100911053Slim Hose mit Coating York - master wine dark,A37110910001Dunkle Businesshose Britt - Black,A37091111068Loose Fit Coloured Denim Rob - master wine authentic wash,A37100910001Slim Hose mit Coating York - Black
A37096410001Coated Slim Fit Hose Yulius - Black,,A37096410109Coated Slim Fit Hose Yulius - night blue,A37100910001Slim Hose mit Coating York - Black,A37100911053Slim Hose mit Coating York - master wine dark,A30038120767Melierter Doubleface Pulli - cyber grey melange,A37100811054Slim Hose mit Coating York - dark pacific blue,A31158930767Wording und Pailletten Pulli - cyber grey melange
A37100910001Slim Hose mit Coating York - Black,,A37100811017Slim Hose mit Coating York - neo grey,A37100810109Slim Hose mit Coating York - night blue,A37100911053Slim Hose mit Coating York - master wine dark,A37100811054Slim Hose mit Coating York - dark pacific blue,A37108620001Hose im Spitzen Look York - Black,A37089410001Coated Slim Fit Hose Henry - Black
A37089410980Coated Slim Fit Hose Henry - baltic grey,,A37089410001Coated Slim Fit Hose Henry - Black,A37089410108Coated Slim Fit Hose Henry - off white,A37100810109Slim Hose mit Coating York - night blue,A37108511017Hose mit Pailletten Yulius - neo grey,A37100911053Slim Hose mit Coating York - master wine dark,A37100811017Slim Hose mit Coating York - neo grey
A37091111129Loose Fit Coloured Denim Rob - neo grey authentic wash,,A37090311039Softe Crash Denim Bonny - sweatdenim black random bleach,A37103311138Glitzerstreifen Denim Crissi - mid blue random bleach wash,A37090711032Loose Fit Used Denim Rob - dark blue auth. random bleach,A37091111033Loose Fit Coloured Denim Rob - studio rose authentic wash,A37095311074Denim mit Nieten Crissi - authentic mid blue fancy wash,A37091111068Loose Fit Coloured Denim Rob - master wine authentic wash
A37091111033Loose Fit Coloured Denim Rob - studio rose authentic wash,,A37091111068Loose Fit Coloured Denim Rob - master wine authentic wash,A37100210660Washed Casual Fit Denim Jane - dark blue rinsed optic,A37095011102Bequeme Stretchdenim Envy - master wine denim wash,A37100211101Washed Casual Fit Denim Jane - midblue basic wash,A37095011069Bequeme Stretchdenim Envy - dark blue clean wash,A37100011102Dunkle Coloured Denim Emmi - master wine denim wash
A31163210001Shirt mit Pailletten Passe - Black,,A31163210821Shirt mit Pailletten Passe - lunar grey,A31163211151Shirt mit Pailletten Passe - evening blue,UNKNOWN,A31163111151Glitzerndes Layering Shirt - evening blue,A31163010821Shirt mit Glitzerkragen - lunar grey,A25258310109Jacke aus Bändchengarn Fenya - night blue
A31163210821Shirt mit Pailletten Passe - lunar grey,,A31163211151Shirt mit Pailletten Passe - evening blue,A37108511017Hose mit Pailletten Yulius - neo grey,A37108620001Hose im Spitzen Look York - Black,A37110910001Dunkle Businesshose Britt - Black,A31164120108Kurzarm Shirt mit Pailletten - off white,UNKNOWN
